# Sensitivity analysis for OBG early detection project

This notebook is designed to help perform a sensitivity analysis for QALY outcomes across a range of values for several parameters in pathosim.

In [ ]:
import pathosim as inf
import sciris as sc
import numpy as np
import time
import math

##### Hyperparameters

In [ ]:
n = 1                # Number of simulations for each scenario
init_inf = 10        # Number of initially infected people
pop_size = 5000      # Size of the population
n_days = 100         # How many days to run each simulation for
maxcpu = 0.9
maxmem = 0.9
parallelize = False  # Parallelization may still have some compatibility issues with the Events class, so keep this as False for now
store_sims = False
base_ifr = 0.01      # The IFR represented by the default SARS-CoV-2 model in pathosim THIS IS ONLY PROVISIONAL, HOW TO ACTUALLY CALCULATE THIS?
avg_contacts = 50    # Average number of contacts, needed to configure transmission rate THIS HAS TO BE SET BASED ON THE POPULATION MODULE

##### Sensitivity analysis parameters

The sensitivity analysis will be run over all permutations of these parameters

In [ ]:
# Situational parameters
ttd = [4, 7, 10]                           # Time to detection (rounded from [4.2, 7.0, 9.7])
response = ['weak']#, 'medium', 'strong']
ttr = [0]                                  # Time to response

# Pathogen parameters
ifr = [0.004, 0.009, 0.014]                # Infection fatality rate (IFR)
incubation = [0.6, 4.0, 5.6]               # Incubation period (low number gives divide by 0-warnings when running the simulations)
beta = [0.91, 2.3, 4.5]                    # Transmission rate

##### Inputs

Life expectancy table for the UK population (taken from National life tables 2020 to 2022, Office for National Statistics). We use the expected residual life (remaining years) at a given age (from age 0 to 100). This allows us to measure the burden of disease in metrics such as Years of Life Lost (YLL).

In [ ]:
le_UK = inf.LifeExpectancy(
    years_remaining_males=[
        78.57, 77.91, 76.93, 75.94, 74.95, 73.96, 72.96, 71.97, 70.97, 69.98,
        68.98, 67.99, 66.99, 66.00, 65.01, 64.01, 63.02, 62.04, 61.06, 60.08,
        59.11, 58.14, 57.17, 56.20, 55.23, 54.26, 53.29, 52.33, 51.36, 50.39,
        49.43, 48.47, 47.51, 46.55, 45.60, 44.65, 43.69, 42.75, 41.81, 40.86,
        39.93, 39.00, 38.07, 37.14, 36.22, 35.30, 34.39, 33.48, 32.58, 31.69,
        30.80, 29.92, 29.04, 28.17, 27.30, 26.43, 25.58, 24.73, 23.89, 23.05,
        22.23, 21.41, 20.61, 19.81, 19.03, 18.25, 17.48, 16.73, 15.99, 15.26,
        14.54, 13.84, 13.14, 12.46, 11.78, 11.12, 10.47, 9.86, 9.26, 8.68,
        8.12, 7.60, 7.08, 6.60, 6.12, 5.68, 5.25, 4.85, 4.48, 4.14, 3.81,
        3.52, 3.25, 3.00, 2.78, 2.57, 2.40, 2.22, 2.07, 1.91, 1.8
],
    years_remaining_females=[
        82.57, 81.86, 80.88, 79.89, 78.90, 77.90, 76.91, 75.91, 74.92, 73.92,
        72.92, 71.93, 70.93, 69.94, 68.94, 67.95, 66.96, 65.97, 64.98, 63.99,
        63.00, 62.01, 61.03, 60.04, 59.06, 58.07, 57.08, 56.10, 55.12, 54.13,
        53.15, 52.17, 51.19, 50.21, 49.24, 48.27, 47.30, 46.33, 45.36, 44.40,
        43.44, 42.48, 41.53, 40.58, 39.63, 38.68, 37.74, 36.80, 35.87, 34.94,
        34.02, 33.10, 32.18, 31.27, 30.36, 29.46, 28.56, 27.66, 26.78, 25.89,
        25.02, 24.15, 23.29, 22.44, 21.60, 20.76, 19.93, 19.12, 18.31, 17.51,
        16.72, 15.94, 15.17, 14.41, 13.66, 12.92, 12.20, 11.51, 10.83, 10.16,
        9.53, 8.91, 8.32, 7.75, 7.20, 6.67, 6.17, 5.70, 5.26, 4.85, 4.47, 4.11,
        3.78, 3.49, 3.23, 2.97, 2.74, 2.54, 2.35, 2.20, 2.04
])

##### Baseline simulation parameters

In [ ]:
sim_pars = dict(
        use_waning    = True,           
        pop_size      = pop_size,       
        pop_type      = 'behaviour_module',      
        n_days        = n_days,            
        verbose       = 0,             
        rand_seed     = 42,
        burden        = inf.Burden(life_expectancy=le_UK) # this will compute burden/utility statistics and report them in results and summary
    ) 

##### Pathogens

Pathogens are generated in a 3-level dictionary where:
- level 1: fatality rate
- level 2: incubation period
- level 3: transmission rate

In [ ]:
pathogens = {}

# Create pathogens from all permutations of IFR, incubation time and transmission rate
for x in ifr:
    pathogens[x] = {}
    for y in incubation:
        pathogens[x][y] = {}
        for z in beta:
            pathogen = inf.SARS_COV_2(init_inf)
            pathogen.configure_hrql_loss(loss_symptomatic=0.43/365, loss_severe=(0.43+0.5)/365, loss_critical=(0.43+0.6)/365)

            # Configure transmission rate
            pathosim_beta = z * avg_contacts/pop_size
            pathogen.configure_transmission(beta = pathosim_beta) 

            # Configure incubation time
            pathogen.configure_disease_state_durations(exp2inf = dict(dist='lognormal_int', par1=y, par2=(y/4.5)*1.5))

            # Configure IFR
            prognoses_by_age = dict(
                age_cutoffs   = np.array([0,       10,      20,      30,      40,      50,      60,      70,      80,      90,]), 
                sus_ORs       = np.array([0.34,    0.67,    1.00,    1.00,    1.00,    1.00,    1.24,    1.47,    1.47,    1.47]), 
                trans_ORs     = np.array([1.00,    1.00,    1.00,    1.00,    1.00,    1.00,    1.00,    1.00,    1.00,    1.00]), 
                comorbidities = np.array([1.00,    1.00,    1.00,    1.00,    1.00,    1.00,    1.00,    1.00,    1.00,    1.00]), 
                symp_probs    = np.array([0.50,    0.55,    0.60,    0.65,    0.70,    0.75,    0.80,    0.85,    0.90,    0.90]), 
                severe_probs  = np.array([0.00050, 0.00165, 0.00720, 0.02080, 0.03430, 0.07650, 0.13280, 0.20655, 0.24570, 0.24570]), 
                crit_probs    = np.array([0.00003, 0.00008, 0.00036, 0.00104, 0.00216, 0.00933, 0.03639, 0.08923, 0.17420, 0.17420]), 
                death_probs   = (x/base_ifr) * np.array([0.00002, 0.00002, 0.00010, 0.00032, 0.00098, 0.00265, 0.00766, 0.02439, 0.08292, 0.16190]), 
            ) 
            pathogen.configure_prognoses(prognoses_by_age, True)

            pathogens[x][y][z] = pathogen

##### Interventions

In [ ]:
# intervention packages (stored as dict)
packages = {
    "weak"   : [inf.intervention_bucket(strength='weak')],
    "medium" : [inf.intervention_bucket(strength='medium')],
    "strong" : [inf.intervention_bucket(strength='strong')]
}

##### Events

In [ ]:
events = [inf.DelayedTrigger('social distancing', start_delay=1, stop_delay=None, trigger_event='detection')]

##### Run baseline and counterfactual simulations

The results are saved in a 4-level dictionary where the first 3 are the pathogen parameters as detailed above. The last level is the response delay. Set verbose to true if more detailed progress report is desired.

In [ ]:
verbose = True
n_param_sets = len(ifr)*len(incubation)*len(beta)*len(ttr)
start_time = time.time()

result = {}
k=0
for x in ifr:
    result[x] = {}
    for y in incubation:
        result[x][y] = {}
        for z in beta:
            result[x][y][z] = {}
            for t in ttr:
                k+=1
                if verbose:
                    print("\n*******************************************************\n")
                print(f"Starting simulations for parameter set {k} of {n_param_sets}")
                if verbose:
                    print(f"\nIFR: {x}\nIncubation period: {y}\nTransmission rate: {z}\nResponse time: {t}\n")

                # Set up response event according to ttr
                events = [inf.DelayedTrigger('social distancing', start_delay=t, stop_delay=None, trigger_event='detection')]

                # Set up simulations and run baseline results
                cf = inf.CounterfactualMultiSim(sim_pars, pathogens = [pathogens[x][y][z]], events=events, intervention_packages=packages, n_sims=n, maxcpu=maxcpu, maxmem = maxmem, parallelize = parallelize)
                cf.run_baseline(verbose = verbose)

                # Run counterfactual results
                for package in list(packages.keys()):
                    if verbose:
                        print(f"Running counterfactual simulations for response: {package}")
                    cf.run_counterfactual(intervention_package_key = package, detection_times = ttd, store_sims = store_sims, verbose = False)

                result[x][y][z][t] = cf.get_summaries_df()[["seed", "intervention_package", "delay", "cum_deaths", "tot_yll", "tot_hrql_loss"]]

dt = time.time() - start_time
print(f"\nTotal computation time for {n_param_sets} parameter sets and {n} seeds per scenario was {math.floor(dt//3600):02d}:{math.floor((dt%3600)//60):02d}:{math.floor(dt%60):02d}")

The results for a particular pathogen and time to responseparameter set can now be retreived:

In [ ]:
result[ifr[0]][incubation[0]][beta[0]][ttr[0]]